# 1. 좋은 타구(배럴)에 대하여 정의하기
- 메이저리그에서는 배럴을 타구 속도와 발사각으로 정의를 내림

# 2.타자 10명 OPS, 장타율, 출루율 예측
- OPS = 장타율 + 출루율
- 장타율 = 1루타 + (2*2루타) + (3*3루타) + (4*홈런) / 타수[AB]
- 출루율 = (안타[HIT] + 사사구) / (타수[AB] + 사사구 + 희생플라이[SF])
- 사사구 = 볼넷[BB] + 사구[HP]

In [1]:
import pandas as pd
pd.set_option('display.max_columns',29)

# 데이터 셋 파일명 설명

- 평가데이터 -> test
- 경기일정 -> game
- 타자 -> hitter
- 선수 -> player
- 타구 -> batting
- 팀 -> team

# 데이터셋 설명

In [2]:
team = pd.read_csv('data/team.csv', encoding='cp949')
team.sample(5)

# T_ID : T팀코드(선수 소속팀의 고유 코드 값)

,SEASON_ID,T_ID,T_NM
8,2016,SS,삼성
47,2020,SK,SK
12,2017,KT,KT
45,2020,NC,NC
54,2021,LT,롯데


In [3]:
game = pd.read_csv('data/2021_game.csv',encoding='cp949')
game.sample(5)

# G_ID : 경기코드
# STADIUM : 해당구장

,G_ID,GDAY_DS,VISIT_KEY,HOME_KEY,HEADER_NO,GWEEK,STADIUM,ACG
201,20210523LGSK0,20210523.0,LG,SK,0,일,문학,1
24,20210409SKLG0,20210409.0,SK,LG,0,금,잠실,1
175,20210516HHWO0,20210516.0,HH,WO,0,일,고척,1
423,20210818WOLT0,20210818.0,WO,LT,0,수,사직,2
712,NaN,NaN,WO,SS,0,NaN,대구,2


In [4]:
hitter_2018 = pd.read_csv('data/2018_hitter.csv')
hitter_2019 = pd.read_csv('data/2019_hitter.csv')
hitter_2020 = pd.read_csv('data/2020_hitter.csv')
hitter_2021 = pd.read_csv('data/2021_hitter.csv')

hitter = pd.concat([hitter_2018, hitter_2019,
                    hitter_2020, hitter_2021])

hitter.sample(5)

# GYEAR : 연도
# PCODE : 선수코드
# GAMENUM : 경기
# PA : 타석
# AB : 타수
# BA : 타율
# HIT : 안타
# HR : 홈런
# TOTB : 루타
# SLG : 장타율
# SF : 희생플라이
# BB : 볼넷(고의4구(IB) 포함한 수치)
# KK : 삼진
# IB : 고의4구
# HP : 사구
# GD : 병살타

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,SF,BB,KK,IB,HP,GD
76,2020,63339,12,5,4,0.000,0,0,0,0.000,0,0,1,0,0,0
47,2019,62907,125,526,468,0.344,161,1,203,0.434,6,41,40,1,9,10
49,2019,62921,6,9,9,0.111,1,0,2,0.222,0,0,5,0,0,1
130,2021,65515,21,35,33,0.121,4,2,10,0.303,0,2,18,0,0,0
223,2018,76812,130,471,407,0.329,134,17,209,0.514,5,43,65,0,12,13


In [5]:
player_2018 = pd.read_csv('data/2018_player.csv', encoding='cp949')
player_2019 = pd.read_csv('data/2019_player.csv', encoding='cp949')
player_2020 = pd.read_csv('data/2020_player.csv', encoding='cp949')
player_2021 = pd.read_csv('data/2021_player.csv', encoding='cp949')

player = pd.concat([player_2018, player_2019,
                    player_2020, player_2021])

player.sample(5)

# GYEAR : 시즌
# PCODE : 선수코드
# NAME : 선수명
# T_ID : 팀코드(선수 소속팀의 고유 코드 값)
# POSITION : 포지션 투 -> 투수, 포 -> 포수, 내 -> 내야수, 외 -> 외야수
# AGE_VA : 나이
# MONEY : 연봉

,GYEAR,PCODE,NAME,T_ID,POSITION,AGE_VA,MONEY
123,2021,51834,르위키,SK,투,29,550000달러
566,2021,74540,강민호,SS,포,35,50000만원
77,2021,51234,로켓,OB,투,27,493000달러
648,2018,99563,김사율,KT,투,38,15000만원
619,2019,78756,오선진,HH,내,30,7500만원


In [6]:
batting_2018 = pd.read_csv('data/2018_batting.csv', encoding='cp949')
batting_2019 = pd.read_csv('data/2019_batting.csv', encoding='cp949')
batting_2020 = pd.read_csv('data/2020_batting.csv', encoding='cp949')
batting_2021 = pd.read_csv('data/2021_batting.csv', encoding='cp949')

batting = pd.concat([batting_2018, batting_2019,
                     batting_2020, batting_2021])

batting.sample(5)

# GYEAR : 연도
# G_ID : 경기코드
# PIT_ID : 타구코드
# PCODE : 선수코드
# T_ID : 타자 팀 코드
# INN : 이닝
# HIT_VEL : 타구속도(km/h)
# HIT_ANG_VER : 발사각도
# HIT_RESULT : 타석결과 - 자세한 설명은 아래에서
# PIT_VEL : 상대투수 타구구속(km/h) 
# STADIUM : 해당구장

,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
10033,2021,20210528OBSS0,210528_193451,75566,SS,3,148.52,0.1,2루타,138.10,대구
25647,2019,20190814WOLG0,190814_213536,78361,LG,9,122.11,79.2,플라이,135.64,잠실
20232,2019,20190707LTWO0,190707_190158,78513,LT,4,160.79,-0.3,1루타,142.41,고척
10480,2021,20210530KTHT0,210530_150859,64868,KT,5,120.34,-2.4,땅볼아웃,141.96,광주
30993,2018,20180920NCHT0,180920_210649,76849,HT,6,154.91,-6.3,땅볼아웃,138.73,광주


# HIT_RESULT : 타석결과

- 1 -> 1루타
- 2 -> 2루타
- 3 -> 3루타
- 4 -> 내야안타(1루타)
- 5 -> 땅볼아웃
- 6 -> 번트아웃
- 7 -> 번트안타
- 8 -> 병살타
- 9 -> 삼중살타
- 10 -> 야수선택
- 11 -> 인필드플라이
- 12 -> 직선타
- 13 -> 파울플라이
- 14 -> 플라이
- 15 -> 홈런
- 16 -> 희생번트
- 17 -> 희생플라이

In [7]:
test = pd.read_excel('data/test.xlsx')
display(test)
submission = test.iloc[1:11,:]
PCODE = [76232, 68050, 75847, 67341, 79192, 
         78224, 78513, 76290, 79215, 67872]
NAME = ['양의지', '강백호',' 최정','이정후','채은성',
        '김재환','전준우','김현수','박건우','로맥']

submission['PCODE'] = PCODE
submission['NAME'] = NAME
submission = submission.fillna(0)
display(submission)

# submission : 제출 양식

,NO.,PCODE,OPS,장타율,출루율
0,NaN,(예시),1.09,0.68,0.417
1,1,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN
5,5,NaN,NaN,NaN,NaN
6,6,NaN,NaN,NaN,NaN
7,7,NaN,NaN,NaN,NaN
8,8,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN


<ipython-input-7-7c7999d21744>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['PCODE'] = PCODE
<ipython-input-7-7c7999d21744>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['NAME'] = NAME


,NO.,PCODE,OPS,장타율,출루율,NAME
1,1,76232,0.0,0.0,0.0,양의지
2,2,68050,0.0,0.0,0.0,강백호
3,3,75847,0.0,0.0,0.0,최정
4,4,67341,0.0,0.0,0.0,이정후
5,5,79192,0.0,0.0,0.0,채은성
6,6,78224,0.0,0.0,0.0,김재환
7,7,78513,0.0,0.0,0.0,전준우
8,8,76290,0.0,0.0,0.0,김현수
9,9,79215,0.0,0.0,0.0,박건우
10,10,67872,0.0,0.0,0.0,로맥


# 컬럼명 바꾸기

In [8]:
hitter.columns = ['연도','선수코드','경기','타석',
                  '타수','타율','안타','홈런',
                  '루타','장타율','희생플라이','볼넷',
                  '삼진','고의사구','사구','병살타']

player.columns = ['연도','선수코드','선수명','팀코드',
                  '포지션','나이','연봉']

batting.columns = ['연도','경기코드','타구코드','선수코드',
                   '팀코드','이닝','타구속도','발사각도',
                   '타석결과','상대투수 타구구속','구장']

batting = batting[['연도','선수코드','경기코드','타구코드',
                  '팀코드','이닝','타구속도','발사각도',
                  '타석결과','상대투수 타구구속','구장']]

In [9]:
hitter.shape, player.shape, batting.shape

((1098, 16), (2639, 7), (120745, 11))

In [10]:
hitter['선수코드'].nunique(), player['선수코드'].nunique(), batting['선수코드'].nunique()

(453, 1066, 437)

# train 데이터 생성

In [11]:
a = pd.merge(batting,player)
train = pd.merge(a, hitter)
train.shape

(120129, 29)

In [12]:
def year(data):
    data = str(data)
    return int(data[:8])

train['날짜'] = train['경기코드'].apply(lambda x: year(x))

# train['날짜'] = train['경기코드'].str[0:8]

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120129 entries, 0 to 120128
Data columns (total 30 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   연도         120129 non-null  int64  
 1   선수코드       120129 non-null  int64  
 2   경기코드       120129 non-null  object 
 3   타구코드       120129 non-null  object 
 4   팀코드        120129 non-null  object 
 5   이닝         120129 non-null  int64  
 6   타구속도       120129 non-null  float64
 7   발사각도       120129 non-null  float64
 8   타석결과       120129 non-null  object 
 9   상대투수 타구구속  120129 non-null  float64
 10  구장         120129 non-null  object 
 11  선수명        120129 non-null  object 
 12  포지션        120129 non-null  object 
 13  나이         120129 non-null  int64  
 14  연봉         120112 non-null  object 
 15  경기         120129 non-null  int64  
 16  타석         120129 non-null  int64  
 17  타수         120129 non-null  int64  
 18  타율         120129 non-null  float64
 19  안타         120129 non-n

In [14]:
train[train['연봉'].isnull()]

,연도,선수코드,경기코드,타구코드,팀코드,이닝,타구속도,발사각도,타석결과,상대투수 타구구속,구장,선수명,포지션,나이,...,타석,타수,타율,안타,홈런,루타,장타율,희생플라이,볼넷,삼진,고의사구,사구,병살타,날짜
101517,2020,50802,20200823OBSK0,200823_171749,SK,1,87.22,34.6,플라이,137.62,문학,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200823
101518,2020,50802,20200823OBSK0,200823_182706,SK,4,86.75,79.7,파울플라이,139.93,문학,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200823
101519,2020,50802,20200823OBSK0,200823_190411,SK,6,94.92,78.7,플라이,138.19,문학,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200823
101520,2020,50802,20200823OBSK0,200823_194030,SK,8,130.99,7.0,직선타,133.34,문학,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200823
101521,2020,50802,20200825SKLT0,200825_183333,SK,1,140.72,24.7,플라이,146.92,사직,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200825
101522,2020,50802,20200910SKHH0,200910_184812,SK,2,129.95,58.4,플라이,143.88,대전,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200910
101523,2020,50802,20200911SKHH0,200911_184925,SK,2,148.88,-1.0,1루타,136.24,대전,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200911
101524,2020,50802,20200911SKHH0,200911_200430,SK,6,149.12,24.6,2루타,132.57,대전,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200911
101525,2020,50802,20200911SKHH0,200911_205821,SK,8,116.68,68.5,파울플라이,145.13,대전,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200911
101526,2020,50802,20200912LTSK0,200912_172337,SK,1,127.41,32.6,플라이,132.23,문학,화이트,내,29,...,30,22,0.136,3,1,7,0.318,0,6,1,0,2,0,20200912
